# Decision - Sistema de Matching de Candidatos

## Pós-Graduação em Data Analytics e Machine Learning

### Autor: [Janaína Cazuza](https://www.linkedin.com/in/janainacazuza/)

## 1. Configuração Inicial

In [ ]:
import pandas as pd
import numpy as np
import re
import json
from pprint import pprint
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    RandomizedSearchCV,
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_recall_curve,
    PrecisionRecallDisplay,
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from tqdm.auto import tqdm

# Configurações
plt.style.use("ggplot")
sns.set_palette("husl")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 200)


In [ ]:

# Caminho do arquivo JSON
caminho_arquivo = "../data/raw/applicants.json"

# Abrir o arquivo JSON
with open(caminho_arquivo, "r", encoding="utf-8") as f:
    dados_json = json.load(f)

# dados_json é um dicionário, então pegamos apenas os valores (cada valor é um candidato)
lista_candidatos = list(dados_json.values())

# Agora aplicamos o json_normalize nessa lista de candidatos
df_applicants = pd.json_normalize(lista_candidatos)

# Visualizar as colunas e as primeiras linhas
print(df_applicants.columns)
print(df_applicants.head())

In [ ]:
df_applicants.columns


In [ ]:
df_applicants.head(5)

In [ ]:
import json
import pandas as pd

# Caminho do arquivo JSON
caminho_arquivo = "../data/raw/prospects.json"

# Abrir o arquivo JSON
with open(caminho_arquivo, "r", encoding="utf-8") as f:
    dados_json = json.load(f)

# Converter o JSON em DataFrame básico (cada linha é uma vaga com uma lista de prospects)
df_prospects = pd.json_normalize(list(dados_json.values()))

# Explodir a coluna 'prospects' para que cada candidato fique em uma linha separada
df_explodido = df_prospects.explode("prospects").reset_index(drop=True)

# Agora, cada linha da coluna 'prospects' é um dicionário — vamos normalizar essa coluna para separar os campos
df_prospects_normalizado = pd.json_normalize(df_explodido["prospects"])

# Concatenar as colunas da vaga com as informações normalizadas dos candidatos
df_prospects_final = pd.concat(
    [df_explodido.drop(columns=["prospects"]), df_prospects_normalizado], axis=1
)

# Visualizar o resultado
print(df_prospects_final.columns)
print(df_prospects_final.head())


In [ ]:
df_prospects_final.columns

In [ ]:
df_prospects_final.head(5)

In [ ]:
import json
import pandas as pd

# 1. Carrega o arquivo jobs.json
with open("../data/raw/jobs.json", "r", encoding="utf-8") as f:
    jobs_raw = json.load(f)

# 2. Transforma o dicionário em uma lista de dicionários, mantendo a chave como 'codigo_vaga'
jobs_list = [
    {"codigo_vaga": codigo, **conteudo} for codigo, conteudo in jobs_raw.items()
]

# 3. Usa json_normalize para achatar os campos aninhados
df_jobs = pd.json_normalize(jobs_list)

# 4. Visualiza as primeiras linhas
df_jobs.head()


In [ ]:
df_jobs.columns

In [ ]:
df_prospects_final = df_prospects_final.rename(columns={"codigo": "codigo_vaga"})
df_vagas_prospectadas = df_prospects_final.merge(df_jobs, on="codigo_vaga", how="left")


In [ ]:
print("Colunas em df_prospects_final:", df_prospects_final.columns)
print("Colunas em df_jobs:", df_jobs.columns)


In [ ]:
df_applicants.columns


In [ ]:
df_prospects_final["vaga_em_jobs"] = df_prospects_final["codigo_vaga"].isin(
    df_jobs["codigo_vaga"]
)


In [ ]:
df_applicants["informacoes_pessoais.nome"] = (
    df_applicants["informacoes_pessoais.nome"].str.lower().str.strip()
)
df_vagas_prospectadas["nome"] = df_vagas_prospectadas["nome"].str.lower().str.strip()


In [ ]:
df_merged = df_vagas_prospectadas.merge(
    df_applicants, left_on="nome", right_on="informacoes_pessoais.nome", how="left"
)


In [ ]:
df_merged.head()



In [ ]:
df_merged.to_csv(
    "../data/processed/merged_data.csv",
    index=False,
    encoding="utf-8",
)


In [ ]:
# Remover duplicatas, se houver
df_merged = df_merged.drop_duplicates()

# Exemplo de padronização de colunas
df_merged.columns = df_merged.columns.str.strip().str.lower().str.replace(" ", "_")


In [ ]:
print(f"Linhas totais: {len(df_merged)}")
print(f"Vagas únicas: {df_merged['codigo_vaga'].nunique()}")
print(f"Candidatos únicos: {df_merged['nome'].nunique()}")


In [ ]:
import matplotlib.pyplot as plt

# Top 10 vagas com mais candidaturas
top_vagas = df_merged["codigo_vaga"].value_counts().head(10)
print(top_vagas)

plt.figure(figsize=(10, 6))
top_vagas.plot(kind="bar")
plt.title("Top 10 vagas com mais candidaturas")
plt.xlabel("Código da vaga")
plt.ylabel("Número de candidaturas")
plt.show()

# Status dos candidatos
status_counts = df_merged["situacao_candidado"].value_counts()
print(status_counts)

plt.figure(figsize=(8, 5))
status_counts.plot(kind="bar", color="skyblue")
plt.title("Distribuição do status dos candidatos")
plt.xlabel("Status")
plt.ylabel("Quantidade")
plt.show()


In [ ]:
import pandas as pd

# 1. Converter a coluna para string e tratar nulos
df_merged["situacao_candidado"] = (
    df_merged["situacao_candidado"].astype(str).replace("nan", "")
)

# 2. Definir status de sucesso
status_sucesso = [
    "Contratado pela Decision",
    "Contratado como Hunting",
    "Aprovado",
    "Proposta Aceita",
]

# 3. Criar target binário
df_merged["target"] = df_merged["situacao_candidado"].isin(status_sucesso).astype(int)

# 4. Análise de distribuição
print("\nDistribuição do target:")
print(df_merged["target"].value_counts(normalize=True))

# 5. Filtrar dados inconclusivos (opcional)
df_modelo = df_merged[
    ~df_merged["situacao_candidado"].str.contains(
        "avaliação|Encaminhado|Prospect", na=False, case=False
    )
]

# Verificar resultado
print("\nDataFrame após filtro:")
print(df_modelo["situacao_candidado"].value_counts())


In [ ]:
# Status considerados como sucesso (1)
sucesso = [
    "Contratado pela Decision",
    "Contratado como Hunting",
    "Aprovado",
    "Proposta Aceita",
]

# Criando a variável target
df_merged["target"] = df_merged["situacao_candidado"].apply(
    lambda x: 1 if x in sucesso else 0
)


In [ ]:
import pandas as pd

# 1. Definir status de sucesso
status_sucesso = [
    "Contratado pela Decision",
    "Contratado como Hunting",
    "Aprovado",
    "Proposta Aceita",
]

# 2. Criar target binário
df_merged["target"] = df_merged["situacao_candidado"].isin(status_sucesso).astype(int)

# 3. Análise de distribuição
print("\nDistribuição do target:")
print(df_merged["target"].value_counts(normalize=True))

# 4. Filtrar dados inconclusivos (opcional)
df_modelo = df_merged[
    ~df_merged["situacao_candidado"].str.contains("avaliação|Encaminhado|Prospect")
]


In [ ]:
import matplotlib.pyplot as plt

df_merged["target"].value_counts().plot.pie(
    autopct="%.1f%%",
    labels=["Não Contratado", "Contratado"],
    colors=["#ff9999", "#66b3ff"],
)
plt.title("Proporção de Contratações (Target)")
plt.show()


In [ ]:
# Converter para string e substituir NaN por string vazia
df_merged["situacao_candidado"] = (
    df_merged["situacao_candidado"].astype(str).replace("nan", "")
)


In [ ]:
# Agora podemos aplicar o filtro corretamente
df_modelo = df_merged[
    ~df_merged["situacao_candidado"].str.contains(
        "avaliação|Encaminhado|Prospect", na=False
    )
]


In [ ]:
import pandas as pd

# 1. Converter a coluna para string e tratar nulos
df_merged["situacao_candidado"] = (
    df_merged["situacao_candidado"].astype(str).replace("nan", "")
)

# 2. Definir status de sucesso
status_sucesso = [
    "Contratado pela Decision",
    "Contratado como Hunting",
    "Aprovado",
    "Proposta Aceita",
]

# 3. Criar target binário
df_merged["target"] = df_merged["situacao_candidado"].isin(status_sucesso).astype(int)

# 4. Análise de distribuição
print("\nDistribuição do target:")
print(df_merged["target"].value_counts(normalize=True))

# 5. Filtrar dados inconclusivos (opcional)
df_modelo = df_merged[
    ~df_merged["situacao_candidado"].str.contains(
        "avaliação|Encaminhado|Prospect", na=False, case=False
    )
]

# Verificar resultado
print("\nDataFrame após filtro:")
print(df_modelo["situacao_candidado"].value_counts())


In [ ]:
# Colunas categóricas relevantes
cat_features = [
    "modalidade",
    "formacao_e_idiomas.nivel_ingles",
    "formacao_e_idiomas.nivel_espanhol",
    "informacoes_pessoais.sexo",
    "informacoes_pessoais.estado_civil",
]

# Colunas de texto para processamento NLP
text_features = [
    "comentario",
    "cv_pt",
    "informacoes_profissionais.conhecimentos_tecnicos",
]

# Colunas numéricas (precisamos criar algumas)
num_features = [
    # Serão criadas a partir das datas
]


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np


class DateFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, date_cols):
        self.date_cols = date_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.date_cols:
            if col in X.columns:
                # Convert to datetime and extract features
                dates = pd.to_datetime(X[col], errors="coerce")
                X[f"{col}_year"] = dates.dt.year
                X[f"{col}_month"] = dates.dt.month
                X[f"{col}_day"] = dates.dt.day
                X[f"{col}_dayofweek"] = dates.dt.dayofweek
        return X




In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import issparse
import numpy as np
from sklearn.decomposition import TruncatedSVD


# 1. Criar um transformador para converter esparso para denso quando necessário
class SparseToDense(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if issparse(X):
            return X.toarray()
        return X


# 2. Definir as colunas para pré-processamento
cat_features = ["modalidade", "formacao_e_idiomas.nivel_ingles"]
text_features = ["comentario", "cv_pt"]

# 3. Criar o pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ("text1", TfidfVectorizer(max_features=100), "comentario"),
        ("text2", TfidfVectorizer(max_features=50), "cv_pt"),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ],
    remainder="drop",
)

# 4. Pipeline final corrigido
final_pipeline = Pipeline(
    [
        ("preprocess", preprocessor),
        ("to_dense", SparseToDense()),  # Conversão explícita aqui
        ("imputer", SimpleImputer(strategy="constant", fill_value=0)),
        ("smote", SMOTE(random_state=42)),
        (
            "classifier",
            RandomForestClassifier(class_weight="balanced", random_state=42),
        ),
    ]
)

# 5. Preparar e treinar
X = df_merged[cat_features + text_features].fillna("")
y = df_merged["target"]

try:
    final_pipeline.fit(X, y)  # Note que agora usamos final_pipeline, não pipeline
    print("Pipeline treinado com sucesso!")
except Exception as e:
    print(f"Erro: {str(e)}")


In [ ]:
from sklearn.model_selection import cross_validate
import time

# 1. Reduza o número de folds e use n_jobs
start = time.time()
cv_results = cross_validate(
    final_pipeline,
    X,
    y,
    cv=3,  # Reduz de 5 para 3 folds
    n_jobs=-1,  # Usa todos os cores do CPU
    scoring=["f1", "roc_auc"],
    verbose=1,  # Mostra progresso
)
print(f"Tempo total: {time.time() - start:.2f} segundos")

# 2. Métricas rápidas
print("\nF1-Score médio:", cv_results["test_f1"].mean())
print("AUC-ROC médio:", cv_results["test_roc_auc"].mean())


In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
import joblib


In [ ]:

# Carregar os dados
df = pd.read_csv("../data/processed/merged_data.csv")
df.shape, df.columns


In [ ]:

# Identificar colunas categóricas e numéricas
cat_cols = X.select_dtypes(include='object').columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Pipeline de transformação
preprocessor = ColumnTransformer(transformers=[
    ("cat", TargetEncoder(), cat_cols),
    ("num", SimpleImputer(strategy="median"), num_cols)
])

# TF-IDF em texto (se houver campo)
# Exemplo fictício: "descricao_perfil" — pode substituir conforme seu dataset
if "descricao_perfil" in X.columns:
    preprocessor.transformers.append(
        ("txt", TfidfVectorizer(max_features=100), "descricao_perfil")
    )

# Construção do pipeline final
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("scaler", StandardScaler(with_mean=False)),
    ("classifier", RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42))
])


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

print("CLASSIFICATION REPORT:\n", classification_report(y_test, y_pred))
print("ROC AUC: ", roc_auc_score(y_test, y_proba))


In [ ]:

# Exportar o modelo treinado
joblib.dump(clf, "modelo_randomforest_match.pkl")
print("Modelo salvo com sucesso para uso no Streamlit!")
